In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [2]:
df_aaindex = pd.read_csv('../data/aaindex/df_aaindex19.csv')
print(df_aaindex.shape)
df_aaindex.head(1)
tmp = df_aaindex.drop('Unnamed: 0',axis=1).T
aa2val = dict()
for aa, val in zip(tmp.index, tmp.values):
    aa2val[aa]=val

(19, 21)


In [3]:
df_detect_peptide_train = pd.read_csv('../data/df_detect_peptide_train.csv')
test = pd.read_csv('../data/df_detect_peptide_test.csv')
train, val = train_test_split(df_detect_peptide_train, test_size=0.2, random_state=7)

df = pd.concat([train, val, test], axis=0).reset_index(drop=True)

train_idx = df.iloc[:len(train), :].index
val_idx = df.iloc[len(train):len(train)+len(val), :].index
test_idx = df.iloc[len(train)+len(val):, :].index

train.head(1)

,peptide,En,Ec,E1,E2,protein,PEP,ID
595411,K.QELNEPPKQSTSFLVLQEILESEEKGDPNK.P,VYKMLQEKQELNEPP,EEKGDPNKPSGFRSV,QELNEPPKQSTSFLV,EILESEEKGDPNKPS,sp|O00151|PDLI1_HUMAN,QELNEPPKQSTSFLVLQEILESEEKGDPNK,0


# torch gpu

In [4]:
import sys
PATH_TO_REPO = "/home/bis/2021_AIhub/esm/"
sys.path.append(PATH_TO_REPO)

import torch
import esm

from tqdm.notebook import tqdm
from tqdm import tqdm, tqdm_notebook
import time
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import AdamW
# from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import WarmupLinearSchedule as get_linear_schedule_with_warmup

In [5]:
##GPU 사용 시
device = torch.device("cuda")

In [6]:
esm_model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
# esm_model, alphabet = esm.pretrained.esm_msa1b_t12_100M_UR50S()
batch_converter = alphabet.get_batch_converter()

In [7]:
esm_model

ProteinBertModel(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj):

In [8]:
count_parameters(esm_model)

+---------------------------------------+------------+
|                Modules                | Parameters |
+---------------------------------------+------------+
|          embed_tokens.weight          |   42240    |
|    layers.0.self_attn.k_proj.weight   |  1638400   |
|     layers.0.self_attn.k_proj.bias    |    1280    |
|    layers.0.self_attn.v_proj.weight   |  1638400   |
|     layers.0.self_attn.v_proj.bias    |    1280    |
|    layers.0.self_attn.q_proj.weight   |  1638400   |
|     layers.0.self_attn.q_proj.bias    |    1280    |
|   layers.0.self_attn.out_proj.weight  |  1638400   |
|    layers.0.self_attn.out_proj.bias   |    1280    |
|  layers.0.self_attn_layer_norm.weight |    1280    |
|   layers.0.self_attn_layer_norm.bias  |    1280    |
|          layers.0.fc1.weight          |  6553600   |
|           layers.0.fc1.bias           |    5120    |
|          layers.0.fc2.weight          |  6553600   |
|           layers.0.fc2.bias           |    1280    |
|    layer

652359094

In [ ]:
652,359,094

In [9]:
# for child in esm_model.children():
#     for param in child.parameters():
#         param.requires_grad = False

In [10]:
class ESMDataset(Dataset):
    def __init__(self, datasets, idxes):
        pep_idx, nterm_idx, cterm_idx, m1term_idx, m2term_idx, label_idx = idxes
        pep_data = [(label, seq) for label, seq in zip(datasets[:, label_idx], datasets[:, pep_idx])]
        nterm_data = [(label, seq) for label, seq in zip(datasets[:, label_idx], datasets[:, nterm_idx])]
        cterm_data = [(label, seq) for label, seq in zip(datasets[:, label_idx], datasets[:, cterm_idx])]
        m1_data = [(label, seq) for label, seq in zip(datasets[:, label_idx], datasets[:, m1term_idx])]
        m2_data = [(label, seq) for label, seq in zip(datasets[:, label_idx], datasets[:, m2term_idx])]
        
        labels, pep_strs, pep_tokens = batch_converter(pep_data)
        _, n_strs, n_tokens = batch_converter(nterm_data)
        _, c_strs, c_tokens = batch_converter(cterm_data)
        _, m1_strs, m1_tokens = batch_converter(m1_data)
        _, m2_strs, m2_tokens = batch_converter(m2_data)

        self.sentences = [pep_tokens, n_tokens, c_tokens, m1_tokens, m2_tokens]
        self.labels = labels

    def __getitem__(self, i):
        return (
                (self.sentences[0][i], ) + (self.sentences[1][i],) \
                + (self.sentences[2][i], ) + (self.sentences[3][i], ) + (self.sentences[4][i], ) \
                + (self.labels[i], )
               )

    def __len__(self):
        return (len(self.labels))

# [PAD] = 1, [MASK] = 21  [CLS] = 0 (special classification token), [SEP] = 2 (seperate segment), Z = 27, '-' = 30, .=29, ,=28
# J 없음
# A 2, B 25, C 23, D 13, E 9, F 18, G 6, H 21, I 12, K 15, L 4, M 20, N 17, 
# O 28, P 14, Q 16, R 10, S 8, T 11, U 26, V 7, W 22, X 24, Y 19, Z 27
# 3 5 없음

In [11]:
## Setting parameters
max_len = 30
batch_size = 4
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  1e-4

In [12]:
s = time.time()

dataset_train = train[['PEP', 'En', 'Ec', 'E1', 'E2', 'ID']].values
dataset_valid = val[['PEP', 'En', 'Ec', 'E1', 'E2', 'ID']].values
dataset_test = test[['PEP', 'En', 'Ec', 'E1', 'E2', 'ID']].values

data_train = ESMDataset(dataset_train, [0, 1, 2, 3, 4, 5])
data_valid = ESMDataset(dataset_valid, [0, 1, 2, 3, 4, 5])

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=48)
valid_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=48)

e = time.time()
print(round(e-s, 2),'sec')

472.22 sec


# model

In [13]:
class ESMClassifier(nn.Module):
    def __init__(self,
                 esm,
                 num_classes=1,
                 params=None):
        
        super(ESMClassifier, self).__init__()
        self.esm = esm
        self.pep_lstm1 = nn.LSTM(input_size=1280, hidden_size=1, batch_first=True)        
        self.ts_lstm1 = nn.LSTM(input_size=1280, hidden_size=1, batch_first=True)
                
        self.fc1 = nn.Linear(5, 1)


#     def gen_attention_mask(self, token_ids, valid_length):
#         attention_mask = torch.zeros_like(token_ids)
#         for i, v in enumerate(valid_length):
#             attention_mask[i][:v] = 1
#         return attention_mask.float()

    def forward(self, token_ids, en_token_ids, ec_token_ids, e1_token_ids, e2_token_ids):
#         attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        pep_embed = self.esm(token_ids, repr_layers=[33])['representations'][33]
        pep_lstm, (pep_hn, __) = self.pep_lstm1(pep_embed)
        
        en_embed = self.esm(en_token_ids, repr_layers=[33])['representations'][33]
        ec_embed = self.esm(ec_token_ids, repr_layers=[33])['representations'][33]
        e1_embed = self.esm(e1_token_ids, repr_layers=[33])['representations'][33]
        e2_embed = self.esm(e2_token_ids, repr_layers=[33])['representations'][33]
        
        en_lstm, (en_hn, __) = self.ts_lstm1(en_embed)
        
        ec_lstm, (ec_hn, __) = self.ts_lstm1(ec_embed)
        
        e1_lstm, (e1_hn, __) = self.ts_lstm1(e1_embed)
        
        e2_lstm, (e2_hn, __) = self.ts_lstm1(e2_embed)
        
        merge = torch.cat([pep_hn[0], en_hn[0], ec_hn[0], e1_hn[0], e2_hn[0]], dim=1)

        merge = self.fc1(merge)        
        out = torch.sigmoid(merge)
        return out

In [14]:
model = ESMClassifier(esm_model).to(device)

In [15]:
model.eval()

ESMClassifier(
  (esm): ProteinBertModel(
    (embed_tokens): Embedding(33, 1280, padding_idx=1)
    (layers): ModuleList(
      (0): TransformerLayer(
        (self_attn): MultiheadAttention(
          (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1280, out_features=5120, bias=True)
        (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      )
      (1): TransformerLayer(
        (self_attn): MultiheadAttention(
          (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (v_proj): Linear(in_fe

In [16]:
count_parameters(model)

+-------------------------------------------+------------+
|                  Modules                  | Parameters |
+-------------------------------------------+------------+
|          esm.embed_tokens.weight          |   42240    |
|    esm.layers.0.self_attn.k_proj.weight   |  1638400   |
|     esm.layers.0.self_attn.k_proj.bias    |    1280    |
|    esm.layers.0.self_attn.v_proj.weight   |  1638400   |
|     esm.layers.0.self_attn.v_proj.bias    |    1280    |
|    esm.layers.0.self_attn.q_proj.weight   |  1638400   |
|     esm.layers.0.self_attn.q_proj.bias    |    1280    |
|   esm.layers.0.self_attn.out_proj.weight  |  1638400   |
|    esm.layers.0.self_attn.out_proj.bias   |    1280    |
|  esm.layers.0.self_attn_layer_norm.weight |    1280    |
|   esm.layers.0.self_attn_layer_norm.bias  |    1280    |
|          esm.layers.0.fc1.weight          |  6553600   |
|           esm.layers.0.fc1.bias           |    5120    |
|          esm.layers.0.fc2.weight          |  6553600  

652369364

In [17]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
loss_fn = nn.BCELoss()
# loss_fn = F.binary_cross_entropy()

t_total = len(train_dataloader) * num_epochs
# warmup_step = int(t_total * warmup_ratio)

# scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps=warmup_step, t_total=t_total)

def calc_accuracy(X,Y):
    train_acc = ((X>0.5)==Y).sum().data.cpu().numpy() / len(Y)
    return train_acc

In [18]:
best_acc = 0
for e in range(num_epochs):
    t0 = time.time()
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (pep_token_ids, n_token_ids, c_token_ids, m1_token_ids, m2_token_ids, label) in enumerate(train_dataloader):
#         print(batch_id, round(time.time()-t0, 2))  # batch8->70000 loop, each 3 sec -> per 7 epoch, 3h
        
        pep_token_ids = pep_token_ids.long().to(device)
        n_token_ids = n_token_ids.long().to(device)
        c_token_ids = c_token_ids.long().to(device)
        m1_token_ids = m1_token_ids.long().to(device)
        m2_token_ids = m2_token_ids.long().to(device)
        label = torch.reshape(label.float(), (-1, 1)).to(device)
        
        pred = model(pep_token_ids, n_token_ids, c_token_ids, m1_token_ids, m2_token_ids)
        loss = F.binary_cross_entropy(pred, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_acc += calc_accuracy(pred, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {} time {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1), round(time.time()-t0, 2)))
        
    print("epoch {} train acc {} time {}".format(e+1, train_acc / (batch_id+1), round(time.time()-t0,2)))
    
    model.eval()
    for batch_id, (pep_token_ids, n_token_ids, c_token_ids, m1_token_ids, m2_token_ids, label) in enumerate(valid_dataloader):
        pep_token_ids = pep_token_ids.long().to(device)
        n_token_ids = n_token_ids.long().to(device)
        c_token_ids = c_token_ids.long().to(device)
        m1_token_ids = m1_token_ids.long().to(device)
        m2_token_ids = m2_token_ids.long().to(device)
        label = label.long().to(device)
        label = torch.reshape(label, (-1, 1))
        pred = model(pep_token_ids, n_token_ids, c_token_ids, m1_token_ids, m2_token_ids)
        
        test_acc += calc_accuracy(pred, label)
    if test_acc > best_acc:
        best_acc=test_acc
        torch.save({"best_acc":best_acc / (batch_id+1) ,"model":model.state_dict()},f'./finetuning_whole.pl')
        print(f"best_acc: {best_acc}")
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1 loss 0.6405693292617798 train acc 0.75 time 4.78
epoch 1 batch id 201 loss 0.7181351184844971 train acc 0.5223880597014925 time 462.63
epoch 1 batch id 401 loss 0.5228497982025146 train acc 0.6022443890274314 time 920.62
epoch 1 batch id 601 loss 0.46038901805877686 train acc 0.632279534109817 time 1383.4
epoch 1 batch id 801 loss 0.6447947025299072 train acc 0.6385767790262172 time 1852.77
epoch 1 batch id 1001 loss 0.5278580784797668 train acc 0.6478521478521478 time 2324.18
epoch 1 batch id 1201 loss 0.6635206937789917 train acc 0.6546627810158201 time 2782.02
epoch 1 batch id 1401 loss 0.6624365448951721 train acc 0.6577444682369736 time 3240.01
epoch 1 batch id 1601 loss 0.5692307949066162 train acc 0.6597439100562149 time 3697.91
epoch 1 batch id 1801 loss 0.4833444356918335 train acc 0.6640755136035535 time 4155.86
epoch 1 batch id 2001 loss 0.6179523468017578 train acc 0.6682908545727136 time 4613.82
epoch 1 batch id 2201 loss 0.721555233001709 train acc 0.67

epoch 1 batch id 18601 loss 0.6980684995651245 train acc 0.575116929197355 time 42662.77
epoch 1 batch id 18801 loss 0.7467469573020935 train acc 0.5744109355885325 time 43119.91
epoch 1 batch id 19001 loss 0.7456659078598022 train acc 0.5735750749960529 time 43576.97
epoch 1 batch id 19201 loss 0.6504814624786377 train acc 0.5729389094318005 time 44033.94
epoch 1 batch id 19401 loss 0.6974238157272339 train acc 0.5722385444049276 time 44490.96
epoch 1 batch id 19601 loss 0.6973432302474976 train acc 0.5717310341309116 time 44948.17
epoch 1 batch id 19801 loss 0.6523370742797852 train acc 0.5709433866976416 time 45405.39
epoch 1 batch id 20001 loss 0.7851259112358093 train acc 0.5702464876756163 time 45862.52
epoch 1 batch id 20201 loss 0.6967732906341553 train acc 0.5692911242017722 time 46319.62
epoch 1 batch id 20401 loss 0.7386776208877563 train acc 0.5687588843684133 time 46776.7
epoch 1 batch id 20601 loss 0.6965295076370239 train acc 0.5680792194553662 time 47233.87
epoch 1 batc

Process Process-37:
Process Process-39:
Process Process-38:
Process Process-40:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/bis/miniconda3/envs/ProtEmbedding/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/home/bis/miniconda3/envs/ProtEmbedding/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/home/bis/miniconda3/envs/ProtEmbedding/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/home/bis/miniconda3/envs/ProtEmbedding/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/home/bis/miniconda3/envs/ProtEmbedding/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
  File "/home/bis/miniconda3/envs/ProtEmbedding/lib/python3.6/multiprocessing/

KeyboardInterrupt: 

In [18]:
best_acc = 0
for e in range(num_epochs):
    t0 = time.time()
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (pep_token_ids, n_token_ids, c_token_ids, m1_token_ids, m2_token_ids, label) in enumerate(train_dataloader):
#         print(batch_id, round(time.time()-t0, 2))  # batch8->70000 loop, each 3 sec -> per 7 epoch, 3h
        
        pep_token_ids = pep_token_ids.long().to(device)
        n_token_ids = n_token_ids.long().to(device)
        c_token_ids = c_token_ids.long().to(device)
        m1_token_ids = m1_token_ids.long().to(device)
        m2_token_ids = m2_token_ids.long().to(device)
        label = torch.reshape(label.float(), (-1, 1)).to(device)
        
        pred = model(pep_token_ids, n_token_ids, c_token_ids, m1_token_ids, m2_token_ids)
        loss = F.binary_cross_entropy(pred, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_acc += calc_accuracy(pred, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {} time {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1), round(time.time()-t0, 2)))
        
    print("epoch {} train acc {} time {}".format(e+1, train_acc / (batch_id+1), round(time.time()-t0,2)))
    
    model.eval()
    for batch_id, (pep_token_ids, n_token_ids, c_token_ids, m1_token_ids, m2_token_ids, label) in enumerate(valid_dataloader):
        pep_token_ids = pep_token_ids.long().to(device)
        n_token_ids = n_token_ids.long().to(device)
        c_token_ids = c_token_ids.long().to(device)
        m1_token_ids = m1_token_ids.long().to(device)
        m2_token_ids = m2_token_ids.long().to(device)
        label = label.long().to(device)
        label = torch.reshape(label, (-1, 1))
        pred = model(pep_token_ids, n_token_ids, c_token_ids, m1_token_ids, m2_token_ids)
        
        test_acc += calc_accuracy(pred, label)
    if test_acc > best_acc:
        best_acc=test_acc
        torch.save({"best_acc":best_acc / (batch_id+1) ,"model":model.state_dict()},f'./finetuning_whole.pl')
        print(f"best_acc: {best_acc}")
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1 loss 0.7029515504837036 train acc 0.25 time 3.23
epoch 1 batch id 201 loss 0.6523497700691223 train acc 0.6355721393034826 time 139.91
epoch 1 batch id 401 loss 0.6511130332946777 train acc 0.6536783042394015 time 509.62
epoch 1 batch id 601 loss 0.672355592250824 train acc 0.6649334442595674 time 1084.9
epoch 1 batch id 801 loss 0.6703794002532959 train acc 0.6707240948813983 time 1660.45
epoch 1 batch id 1001 loss 0.6852197647094727 train acc 0.6750749250749251 time 2236.06
epoch 1 batch id 1201 loss 0.6913191080093384 train acc 0.6824521232306411 time 2811.94
epoch 1 batch id 1401 loss 0.6857815980911255 train acc 0.6847787294789436 time 3387.61
epoch 1 batch id 1601 loss 0.6768355369567871 train acc 0.6862898188632105 time 3963.47
epoch 1 batch id 1801 loss 0.6946740746498108 train acc 0.6868406440866185 time 4539.09
epoch 1 batch id 2001 loss 0.6692252159118652 train acc 0.6881559220389805 time 5115.35
epoch 1 batch id 2201 loss 0.6292107105255127 train acc 0.68

epoch 1 batch id 18601 loss 0.7406469583511353 train acc 0.5422826729745712 time 54566.96
epoch 1 batch id 18801 loss 0.6752915382385254 train acc 0.5418661241423328 time 55170.97
epoch 1 batch id 19001 loss 0.6967687606811523 train acc 0.5414780801010474 time 55775.07
epoch 1 batch id 19201 loss 0.6756535172462463 train acc 0.5410785896567887 time 56379.1
epoch 1 batch id 19401 loss 0.655083417892456 train acc 0.5407453224060615 time 56982.81
epoch 1 batch id 19601 loss 0.6554033756256104 train acc 0.5404188561808071 time 57586.55
epoch 1 batch id 19801 loss 0.6964237689971924 train acc 0.5399853542750366 time 58189.85
epoch 1 batch id 20001 loss 0.6361497044563293 train acc 0.5397355132243388 time 58793.38
epoch 1 batch id 20201 loss 0.6962546706199646 train acc 0.539193604277016 time 59396.85
epoch 1 batch id 20401 loss 0.6571980714797974 train acc 0.5389380422528307 time 60000.37
epoch 1 batch id 20601 loss 0.715394914150238 train acc 0.5385782243580409 time 60603.79
epoch 1 batch 

epoch 1 batch id 37001 loss 0.6868007183074951 train acc 0.5221514283397746 time 108039.73
epoch 1 batch id 37201 loss 0.6934949159622192 train acc 0.5220121770920136 time 108596.11
epoch 1 batch id 37401 loss 0.6870068311691284 train acc 0.5218643886527098 time 109152.39
epoch 1 batch id 37601 loss 0.6934705972671509 train acc 0.5217281455280445 time 109708.56
epoch 1 batch id 37801 loss 0.6810169816017151 train acc 0.521566889764821 time 110263.52
epoch 1 batch id 38001 loss 0.699654221534729 train acc 0.5215027236125365 time 110818.71
epoch 1 batch id 38201 loss 0.6934456825256348 train acc 0.521406507683045 time 111374.94
epoch 1 batch id 38401 loss 0.6934536695480347 train acc 0.5214024374365251 time 111931.05
epoch 1 batch id 38601 loss 0.699426531791687 train acc 0.5212073521411362 time 112486.17
epoch 1 batch id 38801 loss 0.699333906173706 train acc 0.5210948171438881 time 113042.2
epoch 1 batch id 39001 loss 0.6875702142715454 train acc 0.521034717058537 time 113598.57
epoch 

epoch 1 batch id 55201 loss 0.691095769405365 train acc 0.5148344232894332 time 158647.06
epoch 1 batch id 55401 loss 0.6992965936660767 train acc 0.514758307611776 time 159203.5
epoch 1 batch id 55601 loss 0.6911715865135193 train acc 0.5147097174511249 time 159759.96
epoch 1 batch id 55801 loss 0.6872472763061523 train acc 0.5146659558072436 time 160316.33
epoch 1 batch id 56001 loss 0.6987712383270264 train acc 0.5145466152390136 time 160872.78
epoch 1 batch id 56201 loss 0.6931712031364441 train acc 0.5144659347698439 time 161429.28
epoch 1 batch id 56401 loss 0.6931722164154053 train acc 0.5144456658569884 time 161985.72
epoch 1 batch id 56601 loss 0.6931712627410889 train acc 0.514379162912316 time 162542.12
epoch 1 batch id 56801 loss 0.6931695342063904 train acc 0.5143065262935512 time 163098.61
epoch 1 batch id 57001 loss 0.6916094422340393 train acc 0.5142146629006509 time 163654.97
epoch 1 batch id 57201 loss 0.696082592010498 train acc 0.5141147007919442 time 164211.32
epoc

epoch 2 batch id 5401 loss 0.69315105676651 train acc 0.5009026106276615 time 15030.23
epoch 2 batch id 5601 loss 0.6949414014816284 train acc 0.5003347616497054 time 15587.27
epoch 2 batch id 5801 loss 0.6931489706039429 train acc 0.49978451991036027 time 16144.14
epoch 2 batch id 6001 loss 0.6928141117095947 train acc 0.4992084652557907 time 16701.18
epoch 2 batch id 6201 loss 0.6931477189064026 train acc 0.49893162393162394 time 17258.28
epoch 2 batch id 6401 loss 0.6931480169296265 train acc 0.4992579284486799 time 17814.29
epoch 2 batch id 6601 loss 0.6937879920005798 train acc 0.499280412058779 time 18371.4
epoch 2 batch id 6801 loss 0.693147599697113 train acc 0.4988604616968093 time 18928.39
epoch 2 batch id 7001 loss 0.6933163404464722 train acc 0.49866090558491644 time 19485.37
epoch 2 batch id 7201 loss 0.6933102607727051 train acc 0.4987501735870018 time 20042.32
epoch 2 batch id 7401 loss 0.692475438117981 train acc 0.4990879610863397 time 20595.54
epoch 2 batch id 7601 lo

Process Process-110:
Process Process-112:
Process Process-111:
Process Process-109:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-113:
  File "/home/bis/miniconda3/envs/ProtEmbedding/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/home/bis/miniconda3/envs/ProtEmbedding/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/home/bis/miniconda3/envs/ProtEmbedding/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/home/bis/miniconda3/envs/ProtEmbedding/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/home/bis/miniconda3/envs/ProtEmbedding/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
  File "/home/bis/miniconda3/envs/ProtEmbedding/lib/p

KeyboardInterrupt: 

In [23]:
(len(train_dataloader) / 8) * (8.18-5.29) / 3600  # 1210금요일 확인

6.817590277777777